In [1]:
import pandas as pd
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
df = pd.read_csv("ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
df['movieIdCat'] = df.movieId.astype('category').cat.codes.values
df['userIdCat'] = df.userId.astype('category').cat.codes.values
df.head()

,userId,movieId,rating,timestamp,movieIdCat,userIdCat
0,1,1193,5,978300760,1104,0
1,1,661,3,978302109,639,0
2,1,914,3,978301968,853,0
3,1,3408,4,978300275,3177,0
4,1,2355,5,978824291,2162,0


In [3]:
idx = df.groupby('userId').apply(lambda x: x.sample(frac=0.8, random_state = 0)).index.get_level_values(1)

train = df.iloc[idx, :].reset_index(drop = True)
test = df.drop(idx).reset_index(drop = True)

In [5]:
n_users, n_movies = len(df.userIdCat.unique()), len(df.movieIdCat.unique())
n_latent_factors = 3
n_latent_factors_user = 5
n_latent_factors_movie = 8

In [9]:
class Network(nn.Module):
    def __init__(self, n_movies, n_latent_factors_movie, n_users, n_latent_factors_user):
        super().__init__()
        self.embedding_m = nn.Embedding(n_movies+1, n_latent_factors_movie)
        self.embedding_u = nn.Embedding(n_users+1, n_latent_factors_user)

        self.lin1 = nn.Linear(13, 200)
        self.lin2 = nn.Linear(200, 80)
        self.lin3 = nn.Linear(80, 50)
        self.lin4 = nn.Linear(50, 20)
        self.lin5 = nn.Linear(20, 1)
        self.relu = nn.ReLU()
        self.dropout = nn. Dropout(0.1)
        
    def forward(self, x, y):

        m_emb = self.embedding_m(x)
        u_emb = self.embedding_u(y)
        com = torch.cat((m_emb, u_emb), 1)
        res = self.lin1(com)
        res = self.relu(res)
        res = self.dropout(res)
        res = self.lin2(res)
        res = self.relu(res)
        res = self.dropout(res)
        res = self.lin3(res)
        res = self.relu(res)
        res = self.dropout(res)
        res = self.lin4(res)
        res = self.relu(res)
        res = self.dropout(res)
        res = self.lin5(res)
        
        return res

In [10]:
# !pickgpu 7

In [11]:
uin = torch.tensor(train.userIdCat.values).long().cuda()
moin = torch.tensor(train.movieIdCat.values).long().cuda()
out =  torch.tensor(train.rating.values).long().cuda()

model = Network(n_movies, n_latent_factors_movie, n_users, n_latent_factors_user)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay= 0.0001)
model.train()
epochs = 2000
errors = []
criterion = nn.L1Loss()
for epoch in range(epochs):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(moin, uin)

    loss = criterion(y_pred.squeeze().float(),out.float())
    errors.append(loss.item())
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 3.766148805618286
Epoch 1: train loss: 3.7354164123535156
Epoch 2: train loss: 3.700232744216919
Epoch 3: train loss: 3.659844160079956
Epoch 4: train loss: 3.6135575771331787
Epoch 5: train loss: 3.5601823329925537
Epoch 6: train loss: 3.498457193374634
Epoch 7: train loss: 3.427135705947876
Epoch 8: train loss: 3.3451426029205322
Epoch 9: train loss: 3.250765562057495
Epoch 10: train loss: 3.141899347305298
Epoch 11: train loss: 3.01674222946167
Epoch 12: train loss: 2.8740997314453125
Epoch 13: train loss: 2.713160753250122
Epoch 14: train loss: 2.5338220596313477
Epoch 15: train loss: 2.3349387645721436
Epoch 16: train loss: 2.117828845977783
Epoch 17: train loss: 1.887163758277893
Epoch 18: train loss: 1.6518291234970093
Epoch 19: train loss: 1.431620478630066
Epoch 20: train loss: 1.258760929107666
Epoch 21: train loss: 1.172308087348938
Epoch 22: train loss: 1.1952606439590454
Epoch 23: train loss: 1.3016220331192017
Epoch 24: train loss: 1.4256922006607056


Epoch 200: train loss: 0.9910435080528259
Epoch 201: train loss: 0.9903470873832703
Epoch 202: train loss: 0.990023136138916
Epoch 203: train loss: 0.99049973487854
Epoch 204: train loss: 0.9898866415023804
Epoch 205: train loss: 0.9893985986709595
Epoch 206: train loss: 0.9888726472854614
Epoch 207: train loss: 0.9885883331298828
Epoch 208: train loss: 0.9880170822143555
Epoch 209: train loss: 0.9878482222557068
Epoch 210: train loss: 0.9882546663284302
Epoch 211: train loss: 0.9872698187828064
Epoch 212: train loss: 0.9876919984817505
Epoch 213: train loss: 0.9855415225028992
Epoch 214: train loss: 0.9863256216049194
Epoch 215: train loss: 0.985617995262146
Epoch 216: train loss: 0.9854127168655396
Epoch 217: train loss: 0.9858022928237915
Epoch 218: train loss: 0.9849465489387512
Epoch 219: train loss: 0.9842333793640137
Epoch 220: train loss: 0.9842937588691711
Epoch 221: train loss: 0.9839208722114563
Epoch 222: train loss: 0.9838982224464417
Epoch 223: train loss: 0.9823727607727

Epoch 398: train loss: 0.9346501231193542
Epoch 399: train loss: 0.9341038465499878
Epoch 400: train loss: 0.9338788986206055
Epoch 401: train loss: 0.9339330792427063
Epoch 402: train loss: 0.9335679411888123
Epoch 403: train loss: 0.9328808784484863
Epoch 404: train loss: 0.9326488971710205
Epoch 405: train loss: 0.9324449896812439
Epoch 406: train loss: 0.9324544668197632
Epoch 407: train loss: 0.9315348267555237
Epoch 408: train loss: 0.9317287802696228
Epoch 409: train loss: 0.9310141205787659
Epoch 410: train loss: 0.9315198659896851
Epoch 411: train loss: 0.9311258792877197
Epoch 412: train loss: 0.9306558966636658
Epoch 413: train loss: 0.9301113486289978
Epoch 414: train loss: 0.9296546578407288
Epoch 415: train loss: 0.9301071166992188
Epoch 416: train loss: 0.9292166829109192
Epoch 417: train loss: 0.9297637939453125
Epoch 418: train loss: 0.9287779331207275
Epoch 419: train loss: 0.9287789463996887
Epoch 420: train loss: 0.928575336933136
Epoch 421: train loss: 0.9275963306

Epoch 596: train loss: 0.8865623474121094
Epoch 597: train loss: 0.8868972659111023
Epoch 598: train loss: 0.8859747648239136
Epoch 599: train loss: 0.8861514329910278
Epoch 600: train loss: 0.8862234354019165
Epoch 601: train loss: 0.8865970373153687
Epoch 602: train loss: 0.8857471942901611
Epoch 603: train loss: 0.885180652141571
Epoch 604: train loss: 0.8859896063804626
Epoch 605: train loss: 0.8848571181297302
Epoch 606: train loss: 0.8852157592773438
Epoch 607: train loss: 0.8842506408691406
Epoch 608: train loss: 0.8840548992156982
Epoch 609: train loss: 0.8838655948638916
Epoch 610: train loss: 0.8833321928977966
Epoch 611: train loss: 0.8841163516044617
Epoch 612: train loss: 0.8833232522010803
Epoch 613: train loss: 0.8837756514549255
Epoch 614: train loss: 0.8831397294998169
Epoch 615: train loss: 0.8828902840614319
Epoch 616: train loss: 0.8837786316871643
Epoch 617: train loss: 0.8828586935997009
Epoch 618: train loss: 0.8819243907928467
Epoch 619: train loss: 0.8818743824

Epoch 794: train loss: 0.8588976860046387
Epoch 795: train loss: 0.8586576581001282
Epoch 796: train loss: 0.8585212230682373
Epoch 797: train loss: 0.8578365445137024
Epoch 798: train loss: 0.8578678369522095
Epoch 799: train loss: 0.8573806285858154
Epoch 800: train loss: 0.8580658435821533
Epoch 801: train loss: 0.8575142025947571
Epoch 802: train loss: 0.8572975397109985
Epoch 803: train loss: 0.8574376702308655
Epoch 804: train loss: 0.8565758466720581
Epoch 805: train loss: 0.8567678928375244
Epoch 806: train loss: 0.8568585515022278
Epoch 807: train loss: 0.8568171858787537
Epoch 808: train loss: 0.8564252257347107
Epoch 809: train loss: 0.855866014957428
Epoch 810: train loss: 0.8564833402633667
Epoch 811: train loss: 0.8566352128982544
Epoch 812: train loss: 0.8555553555488586
Epoch 813: train loss: 0.8556089401245117
Epoch 814: train loss: 0.8554184436798096
Epoch 815: train loss: 0.8559678196907043
Epoch 816: train loss: 0.8547782301902771
Epoch 817: train loss: 0.8554273247

...

Epoch 1184: train loss: 0.8020749092102051
Epoch 1185: train loss: 0.8012387156486511
Epoch 1186: train loss: 0.8015164732933044
Epoch 1187: train loss: 0.8012805581092834
Epoch 1188: train loss: 0.8021226525306702
Epoch 1189: train loss: 0.8014188408851624
Epoch 1190: train loss: 0.8015398979187012
Epoch 1191: train loss: 0.801098108291626
Epoch 1192: train loss: 0.80096036195755
Epoch 1193: train loss: 0.8009509444236755
Epoch 1194: train loss: 0.8015203475952148
Epoch 1195: train loss: 0.8008288145065308
Epoch 1196: train loss: 0.8002798557281494
Epoch 1197: train loss: 0.8006868362426758
Epoch 1198: train loss: 0.7999071478843689
Epoch 1199: train loss: 0.800258219242096
Epoch 1200: train loss: 0.8002718091011047
Epoch 1201: train loss: 0.8005651235580444
Epoch 1202: train loss: 0.7996394634246826
Epoch 1203: train loss: 0.8000035285949707
Epoch 1204: train loss: 0.7996786832809448
Epoch 1205: train loss: 0.7992231249809265
Epoch 1206: train loss: 0.8004103899002075
Epoch 1207: tra

Epoch 1376: train loss: 0.7842391133308411
Epoch 1377: train loss: 0.7839873433113098
Epoch 1378: train loss: 0.783164918422699
Epoch 1379: train loss: 0.7835405468940735
Epoch 1380: train loss: 0.783829927444458
Epoch 1381: train loss: 0.7841142416000366
Epoch 1382: train loss: 0.7831225991249084
Epoch 1383: train loss: 0.7841551899909973
Epoch 1384: train loss: 0.7838932871818542
Epoch 1385: train loss: 0.7836940288543701
Epoch 1386: train loss: 0.782917320728302
Epoch 1387: train loss: 0.7831754088401794
Epoch 1388: train loss: 0.7835087776184082
Epoch 1389: train loss: 0.7834047079086304
Epoch 1390: train loss: 0.7833272218704224
Epoch 1391: train loss: 0.7839197516441345
Epoch 1392: train loss: 0.7820904850959778
Epoch 1393: train loss: 0.782770574092865
Epoch 1394: train loss: 0.7824702858924866
Epoch 1395: train loss: 0.7826172709465027
Epoch 1396: train loss: 0.7827175259590149
Epoch 1397: train loss: 0.782882809638977
Epoch 1398: train loss: 0.7826213240623474
Epoch 1399: trai

Epoch 1568: train loss: 0.770662784576416
Epoch 1569: train loss: 0.7709463238716125
Epoch 1570: train loss: 0.7713378667831421
Epoch 1571: train loss: 0.7708612084388733
Epoch 1572: train loss: 0.7703925371170044
Epoch 1573: train loss: 0.7706425786018372
Epoch 1574: train loss: 0.7702545523643494
Epoch 1575: train loss: 0.7701841592788696
Epoch 1576: train loss: 0.7701174020767212
Epoch 1577: train loss: 0.7692912817001343
Epoch 1578: train loss: 0.7705177068710327
Epoch 1579: train loss: 0.7705004215240479
Epoch 1580: train loss: 0.7705134749412537
Epoch 1581: train loss: 0.7698944211006165
Epoch 1582: train loss: 0.7699933052062988
Epoch 1583: train loss: 0.7701510787010193
Epoch 1584: train loss: 0.7702596187591553
Epoch 1585: train loss: 0.7699832916259766
Epoch 1586: train loss: 0.7692247033119202
Epoch 1587: train loss: 0.7692475318908691
Epoch 1588: train loss: 0.7701194286346436
Epoch 1589: train loss: 0.769612193107605
Epoch 1590: train loss: 0.7689933776855469
Epoch 1591: t

Epoch 1760: train loss: 0.7603727579116821
Epoch 1761: train loss: 0.760226309299469
Epoch 1762: train loss: 0.760787844657898
Epoch 1763: train loss: 0.7600641250610352
Epoch 1764: train loss: 0.760696291923523
Epoch 1765: train loss: 0.7602638006210327
Epoch 1766: train loss: 0.760431706905365
Epoch 1767: train loss: 0.7601416707038879
Epoch 1768: train loss: 0.7615787386894226
Epoch 1769: train loss: 0.7594478726387024
Epoch 1770: train loss: 0.7602465748786926
Epoch 1771: train loss: 0.7601361870765686
Epoch 1772: train loss: 0.7603459358215332
Epoch 1773: train loss: 0.7596668004989624
Epoch 1774: train loss: 0.7598447203636169
Epoch 1775: train loss: 0.7599672079086304
Epoch 1776: train loss: 0.7596375942230225
Epoch 1777: train loss: 0.7593364715576172
Epoch 1778: train loss: 0.7599220871925354
Epoch 1779: train loss: 0.7593362927436829
Epoch 1780: train loss: 0.7598981857299805
Epoch 1781: train loss: 0.7591469287872314
Epoch 1782: train loss: 0.7589566707611084
Epoch 1783: tra

Epoch 1952: train loss: 0.7501840591430664
Epoch 1953: train loss: 0.7500168681144714
Epoch 1954: train loss: 0.751037061214447
Epoch 1955: train loss: 0.7510544657707214
Epoch 1956: train loss: 0.7509552240371704
Epoch 1957: train loss: 0.7507730722427368
Epoch 1958: train loss: 0.7504874467849731
Epoch 1959: train loss: 0.7500722408294678
Epoch 1960: train loss: 0.7498276829719543
Epoch 1961: train loss: 0.7496412992477417
Epoch 1962: train loss: 0.7503666281700134
Epoch 1963: train loss: 0.7501276731491089
Epoch 1964: train loss: 0.7495028972625732
Epoch 1965: train loss: 0.7496060132980347
Epoch 1966: train loss: 0.7500076293945312
Epoch 1967: train loss: 0.7493695020675659
Epoch 1968: train loss: 0.7506945133209229
Epoch 1969: train loss: 0.7504408955574036
Epoch 1970: train loss: 0.750298261642456
Epoch 1971: train loss: 0.7497023940086365
Epoch 1972: train loss: 0.7493782639503479
Epoch 1973: train loss: 0.749546468257904
Epoch 1974: train loss: 0.7496166229248047
Epoch 1975: tr

In [12]:
torch.save(model.state_dict(),"MDM-Model")

In [13]:
LoadModel = Network(n_movies, n_latent_factors_movie, n_users, n_latent_factors_user)

In [14]:
LoadModel.load_state_dict(torch.load("MDM-Model"))
LoadModel.cuda()
LoadModel.eval()

Network(
  (embedding_m): Embedding(3707, 8)
  (embedding_u): Embedding(6041, 5)
  (lin1): Linear(in_features=13, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=80, bias=True)
  (lin3): Linear(in_features=80, out_features=50, bias=True)
  (lin4): Linear(in_features=50, out_features=20, bias=True)
  (lin5): Linear(in_features=20, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [15]:
test_m = torch.tensor(test.movieIdCat.values).long().to('cuda')
test_u = torch.tensor(test.userIdCat.values).long().to('cuda')
test_pred = LoadModel(test_m, test_u)

In [16]:
import numpy
test_pred.to('cpu')

tensor([[4.4383],
        [3.9978],
        [4.3658],
        ...,
        [4.2153],
        [3.8964],
        [3.8037]], grad_fn=<CopyBackwards>)

In [17]:
FinalResults=test_pred.squeeze().to('cpu').detach().numpy()

In [18]:
test['predicted_rating'] = FinalResults

In [19]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(test['rating'], test['predicted_rating']))

0.7039677480353554


In [20]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(test['predicted_rating'], test['rating']))
print(rmse)

0.904966466712323


In [89]:
pred_df = test[['userId', 'movieId', 'predicted_rating']]

In [91]:
#loading movies data and creating movies dictionary
movies_df = pd.read_csv('movies.csv', usecols=[0,1])
movies_dict = pd.Series(movies_df['title'].values,index=movies_df['movieId']).to_dict()
users_list = pred_df['userId'].unique().tolist()

In [92]:
# recmmendations from ratings predicted by Neural Network
pred_df['predicted_movies'] = pred_df.apply(lambda x: (x['movieId'], x['predicted_rating']), axis=1)
pred_rec_df = pred_df[['userId', 'predicted_movies']].groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')
pred_rec_df['recommendation'] = pred_rec_df['recommendation'].apply(lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))
sorted_recs_dict = pd.Series(pred_rec_df['recommendation'].values,index=pred_rec_df['userId']).to_dict()

/koko/system/anaconda/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
# recmmendations from ratings in dataset
pred_df_t = test[['userId', 'movieId', 'rating']]
pred_df_t['predicted_movies'] = pred_df_t.apply(lambda x: (x['movieId'], x['rating']), axis=1)
pred_rec_df_t = pred_df_t[['userId', 'predicted_movies']].groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')
pred_rec_df_t['recommendation'] = pred_rec_df_t['recommendation'].apply(lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))
sorted_recs_dict_t = pd.Series(pred_rec_df_t['recommendation'].values,index=pred_rec_df_t['userId']).to_dict()

/koko/system/anaconda/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
#recommendation function
def n_rec(user, n, p):
    if p==True:
        n_recs = sorted_recs_dict[user][:n]
    else:
        n_recs = sorted_recs_dict_t[user][:n]
    return [movies_dict[int(x[0])] for x in n_recs]

In [97]:
tp = 0
fp = 0
fn = 0
for user in users_list:
    pred_recs = n_rec(user=user,n=10,p=True)
    rat_recs = n_rec(user=user,n=10,p=False)
    tp = tp + len(list(set(pred_recs) & set(rat_recs)))
    fp = fp + len(list(set(pred_recs) - set(rat_recs)))
    fn = fn + len(list(set(rat_recs) - set(pred_recs)))
precision = tp/float(fp+tp)
recall = tp/float(fn+tp)
fscore = (2*precision*recall)/(precision+recall)

print(precision,recall, fscore)

0.6374516663597865 0.6374516663597865 0.6374516663597865
